In [1]:
import sys
sys.path.append('./paralel_planarity_criterion')

import numpy as np
import random, time

import paralel_methods as pm
from SAT2_solver import SAT2_solver

In [2]:
# 🎲 Generate a random seed
seed = random.randint(0, 2**32 - 1)
print("Using seed:", seed)

# 🌱 Seed NumPy’s RNG
np.random.seed(seed)

Using seed: 2163587941


In [3]:
N = 250 # With N=200 GPU is faster. From 257 We get clEnqueueNDRangeKernel failed: INVALID_WORK_GROUP_SIZE
A = np.random.randint(0, 2, size=(N, N)).astype(np.int8)
B = A.T.copy() 

start_cpu = time.perf_counter()
C_paralel = pm.multiply_or_and(A, B)
end_cpu = time.perf_counter()

print("\nMatriz A:")
print(A)
print("\nMatriz B:")
print(B)
print()
print("----------------------------")
print("Paralel GPU implementation")
print(f"GPU OR-AND Time: {(end_cpu - start_cpu)*1000:.2f} ms")
print("Type of C:", type(C_paralel))
print("\nResultado C (OR-AND):")
print(C_paralel)


# --- CPU: Pure Python ---
solver = SAT2_solver()

start_cpu = time.time()
C_sequential = solver.mult_matrix_or_and(A.tolist(), B.tolist())  # Convert np arrays to lists
end_cpu = time.time()

print()
print("--------------------------")
print("Sequential CPU implementation")
print(f"CPU OR-AND Time: {(end_cpu - start_cpu)*1000:.2f} ms")
print("Type of C:", type(C_sequential))
print("\nResultado C (OR-AND):")
print(np.array(C_sequential, dtype=np.int8))


Matriz A:
[[0 0 1 ... 1 0 1]
 [1 1 0 ... 1 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [1 0 0 ... 1 0 1]
 [1 1 1 ... 1 0 1]
 [0 0 1 ... 1 1 1]]

Matriz B:
[[0 1 0 ... 1 1 0]
 [0 1 1 ... 0 1 0]
 [1 0 0 ... 0 1 1]
 ...
 [1 1 0 ... 1 1 1]
 [0 0 0 ... 0 0 1]
 [1 0 0 ... 1 1 1]]

----------------------------
Paralel GPU implementation
GPU OR-AND Time: 826.52 ms
Type of C: <class 'numpy.ndarray'>

Resultado C (OR-AND):
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]

--------------------------
Sequential CPU implementation
CPU OR-AND Time: 1022.00 ms
Type of C: <class 'list'>

Resultado C (OR-AND):
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]


In [4]:
import numpy as np
import time
from SAT2_solver import SAT2_solver
import pandas as pd

solver = SAT2_solver()

# Parámetros
sizes = list(range(10, 251, 20))  # Tamaños de matriz desde 10 hasta 250 en pasos de 20
repeticiones = 5  # Número de repeticiones por tamaño

# Resultados acumulados
resultados = []

for N in sizes:
    tiempos_gpu = []
    tiempos_cpu = []
    
    for _ in range(repeticiones):
        A = np.random.randint(0, 2, size=(N, N)).astype(np.int8)
        B = A.T.copy()

        # GPU Paralelo (simulado)
        start_gpu = time.perf_counter()
        _ = pm.multiply_or_and(A, B)
        end_gpu = time.perf_counter()
        tiempos_gpu.append((end_gpu - start_gpu) * 1000)

        # CPU Secuencial
        start_cpu = time.perf_counter()
        _ = solver.mult_matrix_or_and(A.tolist(), B.tolist())
        end_cpu = time.perf_counter()
        tiempos_cpu.append((end_cpu - start_cpu) * 1000)

    resultados.append({
        "Tamaño de Matriz": N,
        "Tiempo Promedio GPU (ms)": np.mean(tiempos_gpu),
        "Tiempo Promedio CPU (ms)": np.mean(tiempos_cpu)
    })

# Mostrar resultados como tabla
df = pd.DataFrame(resultados)
print("\n================== RESULTADOS FINALES ==================")
print(df.to_string(index=False))




================== RESULTADOS FINALES ==================
 Tamaño de Matriz  Tiempo Promedio GPU (ms)  Tiempo Promedio CPU (ms)
               10                  25.35326                   0.33418
               30                  25.11590                   2.84716
               50                  25.89030                   9.44762
               70                  25.65770                  24.42990
               90                  26.95348                  50.80702
              110                  33.16448                  97.58426
              130                  37.79830                 150.53772
              150                  42.80240                 230.70142
              170                  66.22094                 325.98122
              190                  64.64436                 455.01690
              210                  91.85060                 636.66090
              230                 102.73146                 824.46190
              250               